# RNN tutorial
- Use tf.nn.dynamic_rnn with MNIST dataset

In [1]:
import time
import numpy as np
import tensorflow as tf

from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data
from pprint import pprint

# 0. Load MNIST data

In [2]:
mnist = input_data.read_data_sets('MNIST_idx3/', one_hot=False)

Extracting MNIST_idx3/train-images-idx3-ubyte.gz
Extracting MNIST_idx3/train-labels-idx1-ubyte.gz
Extracting MNIST_idx3/t10k-images-idx3-ubyte.gz
Extracting MNIST_idx3/t10k-labels-idx1-ubyte.gz


In [3]:
X_train, Y_train = mnist.train.images, mnist.train.labels
X_val, Y_val = mnist.validation.images, mnist.validation.labels
X_test, Y_test = mnist.test.images, mnist.test.labels

In [4]:
dim_X = X_train.shape[1]
pixel_X = int(np.sqrt(dim_X)) # np.sqrt의 출력이 float32이므로, 이를 int 자료형으로 변경

print("Dimension of X: %d (%d x %d)" % (dim_X, pixel_X, pixel_X))

Dimension of X: 784 (28 x 28)


# 1. Parameters

In [5]:
LEARNING_RATE  = 0.001
N_EPOCHS       = 10
BATCH_SIZE     = 128
DISPLAY_STEP   = 50
VAL_EPOCH      = 5

n_input   = pixel_X
n_steps   = pixel_X
n_hidden  = 32
n_classes = 10

## 2. Placeholders

In [6]:
X = tf.placeholder(tf.float32, [None, dim_X])
Y = tf.placeholder(tf.int32, [None,])

In [7]:
print(X.get_shape())

(?, 784)


## reshape

In [8]:
# for RNN I need tensor which is axis=1
reshaped_X = tf.reshape(tensor=X, shape=[-1, n_steps, n_input])
unstacked_X = tf.unstack(value=reshaped_X, num=n_steps, axis=1)

print("X shpae: ", X.get_shape())
print("X reshape: ", reshaped_X.get_shape())

X shpae:  (?, 784)
X reshape:  (?, 28, 28)


## 3. Model

In [9]:
def RNN_dynamic(X, n_steps, n_hidden, n_classes):
    
    reshaped_X = tf.reshape(tensor=X, shape=[-1, n_steps, n_input], name="reshaped")
    
    # Define RNN cell
    rnn_basic_cell = rnn.BasicRNNCell(num_units=n_hidden)
    
    # Get RNN cell last output
    outputs, _ = tf.nn.dynamic_rnn(cell=rnn_basic_cell,
                             inputs=reshaped_X,
                             dtype=tf.float32)
    print(outputs.get_shape())
    last_step = list(outputs.get_shape())[1]
    last_output = outputs[:, last_step -1, :]
    
    # Define weights and biases
    W = tf.get_variable(name="weight",
                       shape=[n_hidden, n_classes],
                       initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable(name="bias",
                        shape=[n_classes],
                        initializer=tf.constant_initializer(0.0))
    
    # Calculate logits
    logits = tf.nn.xw_plus_b(x=last_output, weights=W, biases=b, name="logits")
    
    return logits

In [10]:
# logits: output vector of RNN_static
logits = RNN_dynamic(X, n_steps, n_hidden, n_classes)
print(logits)

(?, 28, 32)
Tensor("logits:0", shape=(?, 10), dtype=float32)


In [11]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=Y))
train_op = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(cost)

In [12]:
correct_prediction = tf.nn.in_top_k(predictions=logits, targets=Y, k=1)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
def make_batch_step(Y, batch_size, allow_small_batch=True):
    num_points = len(Y)
    start_idx = list(range(0, num_points, batch_size))
    end_idx = list(range(batch_size, num_points+1, batch_size))
    
    if allow_small_batch:
        start_idx.append(end_idx[-1] + 1)
        end_idx.append(num_points)
    
    return zip(start_idx, end_idx)

# 4. Run

In [14]:
start_time = time.time()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(N_EPOCHS):
        batch_step = make_batch_step(Y=Y_train,
                                        batch_size=BATCH_SIZE,
                                        allow_small_batch=True)
        step = 0

        # Train
        for start, end in batch_step:
            batch_xs = X_train[start:end]
            batch_ys = Y_train[start:end]
            
            _, cost_batch, accuracy_batch = sess.run([train_op, cost, accuracy],
                                                    feed_dict={X:batch_xs, Y:batch_ys})
            if step % DISPLAY_STEP is 0:
                print("[%3d step / %3d epoch] cost = %.6f, accuracy = %.6f" % (step, epoch, cost_batch, accuracy_batch))
                
            step += 1
        
        # Validation
        if epoch % VAL_EPOCH is 0:
            cost_val, accuracy_val = sess.run([cost, accuracy], feed_dict={X:X_val, Y:Y_val})
            print()
            print("\t[%3d epoch] validation cost = %.6f, validation accuracy = %.6f" % (epoch, cost_val, accuracy_val))
            print()
        
        print()
    
    # Test
    cost_test, accuracy_test = sess.run([cost, accuracy], feed_dict={X: X_test, Y: Y_test})
    print("\n\t[Test] test cost = %.6f, test accuracy = %.6f" % (cost_test, accuracy_test))

[  0 step /   0 epoch] cost = 2.321167, accuracy = 0.046875
[ 50 step /   0 epoch] cost = 1.713754, accuracy = 0.414062
[100 step /   0 epoch] cost = 1.402698, accuracy = 0.546875
[150 step /   0 epoch] cost = 1.293166, accuracy = 0.570312
[200 step /   0 epoch] cost = 1.124235, accuracy = 0.656250
[250 step /   0 epoch] cost = 1.191664, accuracy = 0.570312
[300 step /   0 epoch] cost = 0.893257, accuracy = 0.695312
[350 step /   0 epoch] cost = 0.907208, accuracy = 0.671875
[400 step /   0 epoch] cost = 0.926002, accuracy = 0.656250

	[  0 epoch] validation cost = 0.880710, validation accuracy = 0.724200


[  0 step /   1 epoch] cost = 0.854653, accuracy = 0.726562
[ 50 step /   1 epoch] cost = 0.713805, accuracy = 0.734375
[100 step /   1 epoch] cost = 0.832193, accuracy = 0.718750
[150 step /   1 epoch] cost = 0.835297, accuracy = 0.726562
[200 step /   1 epoch] cost = 0.772626, accuracy = 0.742188
[250 step /   1 epoch] cost = 0.894310, accuracy = 0.687500
[300 step /   1 epoch] co

In [15]:
print(time.time() - start_time, "sec")

78.40251421928406 sec
